<a href="https://colab.research.google.com/github/omkar806/on-device-experiments/blob/main/Copy_of_knowledge_base_graphQL_code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run the below cell to install all the libraries

In [ ]:
!pip install -q langchain openai langchain_openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.1 MB/s eta 0:00:00


#Import all the neccessary libraries using the below cell

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
import openai
import os
from google.colab import userdata
from collections import deque

#Defining a Pydantic Model to extract the exact information for Each key and Validate the json output given by the LLM.

In [ ]:
#Defining a pydantic model for getting the exact strcutured json from the input query as we want
class Candidate(BaseModel):
    entity: str = Field(description="The company name mentioned in the query (e.g., Flipkart, Amazon).")
    parameter: str = Field(description="The performance metric mentioned in the query (e.g., GMV, revenue, profit)")
    startDate: str = Field(description=f"The start date of the time period for which the metric is requested. By default,if the user query does not explicitly mention the start date.Assume Default start date to be : Today's date(is {datetime.today().date()}) minus one year.")
    endDate: str = Field(description=f"The end date of the time period for which the metric is requested.By default If the user query does not explicitly mention the end date. Assume the end date to be : Today's date(is {datetime.today().date()}).")

#Initialising the OpenAI model , Please provide your valid OPENAI_API_KEY in the cell below in the first line

In [ ]:
API_KEY=userdata.get("OPENAI_API_KEY")
model_name = "gpt-3.5-turbo-instruct"
temperature = 0.0
model = OpenAI(openai_api_key=API_KEY,model_name=model_name, temperature=temperature, max_tokens=800)

#Function responsible for taking the user query processing it and Validating it with Pydantic model keys and printing the Json output

In [ ]:
graphQL_template = """

The response should understand a user question and return a graphql query according to it.
The Supabase table under consideration is named as "receipt_radar_insights" and it has the following column_names:
purchaseCategories,competitorReceipts,shoppingDates,spendingCapacity
The response must ONLY contain the code snippet and NOTHING else.
The response must be one single line which contains only the query and must not be assigned to a variable.
Example 1:
Question: what's is previuous month budget ?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{shoppingDates spendingCapacity}

}
  }
    }

Example 2:
Question: What did I spend on last month?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{purchaseCategories}

}
  }
    }

"""

In [ ]:
def bot_response(user_query,response_history):
  try:
    prompt = PromptTemplate(
      template="Answer the user query based on the below examples and template.\n{graphQL_template}\n User Query :\n {query} .\n\nMessage History:\n{history}.\n Above is your previous response.",
      input_variables=["query"],
      partial_variables={"graphQL_template": graphQL_template, "history": "\n".join(str(list(response_history)))},
    )
    input = prompt.format_prompt(query=str(user_query))
    print(input.to_string())
    with get_openai_callback() as cb:
      try:
        result = model.invoke(input.to_string())
        print("printing result")
        print(result)
      except Exception as e:
        print(f"Error {e}")

      try:
        print("saving history")
        response_history.append(result)
        print("Bot: ",list(response_history))
        return list(response_history)
      except Exception as e:
        print(f"Error {e}")
        response_history.append("None")
  except Exception as e:
    print(f"Error : {e}")


#Below cell can be used for interaction , You can test your prompts here

In [ ]:
message_history = deque(maxlen=4)
while True:
  user_query = input("User: ")
  user_query = user_query
  if user_query == "exit":
    break
  bot_response(user_query,message_history)

User: what's is previuous month budget ?
Answer the user query based on the below examples and template.
 

The response should understand a user question and return a graphql query according to it.
The Supabase table under consideration is named as "receipt_radar_insights" and it has the following column_names:
purchaseCategories,competitorReceipts,shoppingDates,spendingCapacity
The response must ONLY contain the code snippet and NOTHING else.
The response must be one single line which contains only the query and must not be assigned to a variable.
Example 1:
Question: whats is previuous month budget ?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{shoppingDates spendingCapacity}

}
  }
    }

Example 2:
Question: What did I spend on last month?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{purchaseCategories}

}
  }
    }


 User Query :
 what's is previuous month budget ? .

Message History:
[
].
 Above is your previous response.


printing result

saving history
Bot:  ['']
